In [8]:
!python -m pip install huggingface_hub
!python -m pip install datasets


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


This was run with python 3.11.9 with a RTX 4070 GPU

In [9]:
from datasets import load_dataset
import torch
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader

from setfit import SetFitModel, SetFitTrainer, Trainer
from sentence_transformers import models, SentenceTransformer
from transformers import BertConfig, AutoTokenizer, AutoModel, BertModel

import time
import numpy as np
import pandas as pd

Get the dataset from Huggingface hub

In [10]:
# dataset from hf_hub
langs = ['java', 'python', 'pharo']
labels = {
    'java': ['summary', 'Ownership', 'Expand', 'usage', 'Pointer', 'deprecation', 'rational'],
    'python': ['Usage', 'Parameters', 'DevelopmentNotes', 'Expand', 'Summary'],
    'pharo': ['Keyimplementationpoints', 'Example', 'Responsibilities', 'Classreferences', 'Intent', 'Keymessages', 'Collaborators']
}
ds = load_dataset('NLBSE/nlbse25-code-comment-classification')
ds = ds.with_format("torch")
ds

DatasetDict({
    java_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 7614
    })
    java_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1725
    })
    python_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1884
    })
    python_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 406
    })
    pharo_train: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 1298
    })
    pharo_test: Dataset({
        features: ['index', 'class', 'comment_sentence', 'partition', 'combo', 'labels'],
        num_rows: 289
    })
})

First the configuration of the java model

In [11]:
conf_java = {
  "_name_or_path": "nreimers/MiniLM-L6-H384-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": None,
  "gradient_checkpointing": False,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 1,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.45.2",
  "type_vocab_size": 2,
  "use_cache": True,
  "vocab_size": 30522
}

The transformer is taken from huggingface hub and the conf_java configuration arguments are applied. The untrained model is then constructed from the transformer and pooling using the SentenceTransformer (SBERT) library.

In [12]:
transformer = models.Transformer("sentence-transformers/all-MiniLM-L6-v2", max_seq_length=256, config_args=conf_java)
pooling = models.Pooling(transformer.get_word_embedding_dimension(), 
    pooling_mode_cls_token=False, pooling_mode_mean_tokens=True, pooling_mode_max_tokens=False, pooling_mode_mean_sqrt_len_tokens=False, pooling_mode_weightedmean_tokens=False, pooling_mode_lasttoken=False, include_prompt=True)
modelMy = SentenceTransformer(modules=[transformer, pooling])
modelMy.save_pretrained('./models/aight-l6-java')

Some weights of the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 were not used when initializing BertModel: ['encoder.layer.1.attention.output.LayerNorm.bias', 'encoder.layer.1.attention.output.LayerNorm.weight', 'encoder.layer.1.attention.output.dense.bias', 'encoder.layer.1.attention.output.dense.weight', 'encoder.layer.1.attention.self.key.bias', 'encoder.layer.1.attention.self.key.weight', 'encoder.layer.1.attention.self.query.bias', 'encoder.layer.1.attention.self.query.weight', 'encoder.layer.1.attention.self.value.bias', 'encoder.layer.1.attention.self.value.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.attention.output.LayerNorm.bias', 'encoder.layer.2.attention.output.LayerNorm.weight', 'encoder.layer.2.attention.output.dense.bias', 'encod

The untrained model is loaded as a SetFitModel, and a trainer is set up using the dataset. The trainer then trains the model, and it is saved.

In [13]:
model = SetFitModel.from_pretrained("./models/aight-l6-java", multi_target_strategy="multi-output",device='cuda', normalize_embeddings=False)
trainer = SetFitTrainer(
    model=model,
    train_dataset=ds['java_train'],
    column_mapping={"combo": "text", "labels": "label"},
    num_epochs=5,
    batch_size=32,
)
trainer.train()
trainer.model.save_pretrained('./models/aight-l6-java-trained')

model_head.pkl not found in D:\Documents\!Colleg\CS440-Final\Finalized\models\aight-l6-java, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
C:\Users\Tea\AppData\Local\Temp\ipykernel_12376\3123015787.py:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
***** Running training *****
  Num unique pairs = 304560
  Batch size = 32
  Num epochs = 5
                                                    
 13%|█▎        | 6278/47590 [1:23:43<22:13, 30.97it/s]

{'embedding_loss': 0.3037, 'grad_norm': 0.5002561211585999, 'learning_rate': 4.2025635637739025e-09, 'epoch': 0.0}


                                                      
 13%|█▎        | 6278/47590 [1:23:44<22:13, 30.97it/s]

{'embedding_loss': 0.2901, 'grad_norm': 0.6191548705101013, 'learning_rate': 2.1012817818869512e-07, 'epoch': 0.01}


                                                      
 13%|█▎        | 6278/47590 [1:23:45<22:13, 30.97it/s]

...

{'embedding_loss': 0.0051, 'grad_norm': 0.06167925149202347, 'learning_rate': 4.202563563773902e-08, 'epoch': 4.99}


                                                      
 13%|█▎        | 6278/47590 [1:40:43<22:13, 30.97it/s]

{'embedding_loss': 0.0054, 'grad_norm': 0.8440632224082947, 'learning_rate': 1.8678060283439564e-08, 'epoch': 5.0}


                                                      
100%|██████████| 47590/47590 [17:01<00:00, 46.57it/s]]


{'train_runtime': 1021.8673, 'train_samples_per_second': 1490.213, 'train_steps_per_second': 46.572, 'train_loss': 0.025664358941615444, 'epoch': 5.0}


Next, the configuration for the python model.

In [14]:
conf_python = {
  "_name_or_path": "nreimers/MiniLM-L6-H384-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": None,
  "gradient_checkpointing": False,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 2,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.45.2",
  "type_vocab_size": 2,
  "use_cache": True,
  "vocab_size": 30522
}

The transformer is taken from huggingface hub and the conf_python configuration arguments are applied. The model is then constructed from the transformer and pooling using the SentenceTransformer (SBERT)library.

In [15]:
transformer = models.Transformer("sentence-transformers/all-MiniLM-L6-v2", max_seq_length=256, config_args=conf_python)
pooling = models.Pooling(transformer.get_word_embedding_dimension(), 
    pooling_mode_cls_token=False, pooling_mode_mean_tokens=True, pooling_mode_max_tokens=False, pooling_mode_mean_sqrt_len_tokens=False, pooling_mode_weightedmean_tokens=False, pooling_mode_lasttoken=False, include_prompt=True)
modelMy = SentenceTransformer(modules=[transformer, pooling])
modelMy.save_pretrained('./models/aight-l6-python')

Some weights of the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 were not used when initializing BertModel: ['encoder.layer.2.attention.output.LayerNorm.bias', 'encoder.layer.2.attention.output.LayerNorm.weight', 'encoder.layer.2.attention.output.dense.bias', 'encoder.layer.2.attention.output.dense.weight', 'encoder.layer.2.attention.self.key.bias', 'encoder.layer.2.attention.self.key.weight', 'encoder.layer.2.attention.self.query.bias', 'encoder.layer.2.attention.self.query.weight', 'encoder.layer.2.attention.self.value.bias', 'encoder.layer.2.attention.self.value.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.attention.output.LayerNorm.bias', 'encoder.layer.3.attention.output.LayerNorm.weight', 'encoder.layer.3.attention.output.dense.bias', 'encod

The untrained model is loaded as a SetFitModel, and a trainer is set up using the dataset. The trainer then trains the model, and it is saved.

In [16]:
model = SetFitModel.from_pretrained("./models/aight-l6-python", multi_target_strategy="multi-output",device='cuda', normalize_embeddings=False)
trainer = SetFitTrainer(
    model=model,
    train_dataset=ds['python_train'],
    column_mapping={"combo": "text", "labels": "label"},
    num_epochs=5,
    batch_size=32,
)
trainer.train()
trainer.model.save_pretrained('./models/aight-l6-python-trained')

model_head.pkl not found in D:\Documents\!Colleg\CS440-Final\Finalized\models\aight-l6-python, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
C:\Users\Tea\AppData\Local\Temp\ipykernel_12376\1205607608.py:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
***** Running training *****
  Num unique pairs = 75360
  Batch size = 32
  Num epochs = 5
                                                      
 13%|█▎        | 6278/47590 [1:41:29<22:13, 30.97it/s]

{'embedding_loss': 0.2599, 'grad_norm': 0.39053067564964294, 'learning_rate': 1.6977928692699493e-08, 'epoch': 0.0}


                                                      
 13%|█▎        | 6278/47590 [1:41:30<22:13, 30.97it/s]

{'embedding_loss': 0.2605, 'grad_norm': 0.47217968106269836, 'learning_rate': 8.488964346349746e-07, 'epoch': 0.02}


                                                      
 13%|█▎        | 6278/47590 [1:41:31<22:13, 30.97it/s]

...

{'embedding_loss': 0.0226, 'grad_norm': 1.2740271091461182, 'learning_rate': 1.4154949514013402e-07, 'epoch': 4.97}


                                                      
 13%|█▎        | 6278/47590 [2:01:40<22:13, 30.97it/s]

{'embedding_loss': 0.0249, 'grad_norm': 0.4734831750392914, 'learning_rate': 4.7183165046711335e-08, 'epoch': 4.99}


                                                      
100%|██████████| 11775/11775 [20:13<00:00,  9.70it/s]]


{'train_runtime': 1213.4355, 'train_samples_per_second': 310.523, 'train_steps_per_second': 9.704, 'train_loss': 0.06802977183821854, 'epoch': 5.0}


The configuration for the pharo model.

In [17]:
conf_pharo = {
  "_name_or_path": "nreimers/MiniLM-L6-H384-uncased",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": None,
  "gradient_checkpointing": False,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.45.2",
  "type_vocab_size": 2,
  "use_cache": True,
  "vocab_size": 30522
}

The transformer is taken from huggingface hub and the conf_pharo configuration arguments are applied. The model is then constructed from the transformer and pooling using the SentenceTransformer (SBERT)library.

In [18]:
transformer = models.Transformer("sentence-transformers/all-MiniLM-L6-v2", max_seq_length=256, config_args=conf_pharo)
pooling = models.Pooling(transformer.get_word_embedding_dimension(), 
    pooling_mode_cls_token=False, pooling_mode_mean_tokens=True, pooling_mode_max_tokens=False, pooling_mode_mean_sqrt_len_tokens=False, pooling_mode_weightedmean_tokens=False, pooling_mode_lasttoken=False, include_prompt=True)
modelMy = SentenceTransformer(modules=[transformer, pooling])
modelMy.save_pretrained('./models/aight-l6-pharo')

Some weights of the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 were not used when initializing BertModel: ['encoder.layer.3.attention.output.LayerNorm.bias', 'encoder.layer.3.attention.output.LayerNorm.weight', 'encoder.layer.3.attention.output.dense.bias', 'encoder.layer.3.attention.output.dense.weight', 'encoder.layer.3.attention.self.key.bias', 'encoder.layer.3.attention.self.key.weight', 'encoder.layer.3.attention.self.query.bias', 'encoder.layer.3.attention.self.query.weight', 'encoder.layer.3.attention.self.value.bias', 'encoder.layer.3.attention.self.value.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.dense.weight', 'encoder.layer.3.output.LayerNorm.bias', 'encoder.layer.3.output.LayerNorm.weight', 'encoder.layer.3.output.dense.bias', 'encoder.layer.3.output.dense.weight', 'encoder.layer.4.attention.output.LayerNorm.bias', 'encoder.layer.4.attention.output.LayerNorm.weight', 'encoder.layer.4.attention.output.dense.bias', 'encod

The untrained model is loaded as a SetFitModel, and a trainer is set up using the dataset. The trainer then trains the model, and it is saved.

In [19]:
model = SetFitModel.from_pretrained("./models/aight-l6-pharo", multi_target_strategy="multi-output",device='cuda', normalize_embeddings=False)
trainer = SetFitTrainer(
    model=model,
    train_dataset=ds['pharo_train'],
    column_mapping={"combo": "text", "labels": "label"},
    num_epochs=10,
    batch_size=32,
)
trainer.train()
trainer.model.save_pretrained('./models/aight-l6-pharo-trained')

model_head.pkl not found in D:\Documents\!Colleg\CS440-Final\Finalized\models\aight-l6-pharo, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
C:\Users\Tea\AppData\Local\Temp\ipykernel_12376\1695209910.py:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
***** Running training *****
  Num unique pairs = 51920
  Batch size = 32
  Num epochs = 10
                                                      
 13%|█▎        | 6278/47590 [2:02:01<22:13, 30.97it/s]

{'embedding_loss': 0.2158, 'grad_norm': 0.36696842312812805, 'learning_rate': 1.2322858903265559e-08, 'epoch': 0.0}


                                                      
 13%|█▎        | 6278/47590 [2:02:06<22:13, 30.97it/s]

{'embedding_loss': 0.2602, 'grad_norm': 0.542107105255127, 'learning_rate': 6.161429451632779e-07, 'epoch': 0.03}


                                                      
 13%|█▎        | 6278/47590 [2:02:12<22:13, 30.97it/s]

...

{'embedding_loss': 0.0122, 'grad_norm': 0.47588911652565, 'learning_rate': 1.0953652358458274e-07, 'epoch': 9.95}


                                                      
 13%|█▎        | 6278/47590 [2:18:57<22:13, 30.97it/s]


{'embedding_loss': 0.0117, 'grad_norm': 0.2696921229362488, 'learning_rate': 4.1076196344218526e-08, 'epoch': 9.98}


                                                      [A
100%|██████████| 16230/16230 [16:58<00:00, 15.94it/s]]


{'train_runtime': 1018.3114, 'train_samples_per_second': 509.864, 'train_steps_per_second': 15.938, 'train_loss': 0.03165035528106155, 'epoch': 10.0}


Now that all three models have been set up and trained, they can be tested and scored. The testing is done bu running each of them through with testing data while capturing the compute and run time, then calculating the total compute, average runtime, and individually the precision, recall, and F1 by category.

In [26]:
total_flops = 0
total_time = 0
scores = []
num = range(3)
langs = ['java', 'python', 'pharo']
for lan in langs:
    print(lan)
    # to load newly trained models leave as is, otherwise change the directory to './models-pretrained/aight-l6-{lan}-trained'
    model = SetFitModel.from_pretrained(f'./models/aight-l6-{lan}-trained')
    with torch.profiler.profile(with_flops=True) as p:
        begin = time.time()
        for i in range(10):
          y_pred = model(ds[f'{lan}_test']['combo']).numpy().T
        total = time.time() - begin
        total_time = total_time + total
    total_flops = total_flops + (sum(k.flops for k in p.key_averages()) / 1e9)
    y_true = np.array(ds[f'{lan}_test']['labels']).T
    for i in range(len(y_pred)):
        assert(len(y_pred[i]) == len(y_true[i]))
        tp = sum([true == pred == 1 for (true,pred) in zip(y_true[i], y_pred[i])])
        tn = sum([true == pred == 0 for (true,pred) in zip(y_true[i], y_pred[i])])
        fp = sum([true == 0 and pred == 1 for (true,pred) in zip(y_true[i], y_pred[i])])
        fn = sum([true == 1 and pred == 0 for (true,pred) in zip(y_true[i], y_pred[i])])
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = (2*tp) / (2*tp + fp + fn)
        scores.append({'lan': lan, 'cat': labels[lan][i],'precision': precision,'recall': recall,'f1': f1})
print("Compute in GFLOPs:", total_flops/10)
print("Avg runtime in seconds:", total_time/10)
scores = pd.DataFrame(scores)
scores

java
python
pharo
Compute in GFLOPs: 458.604438528
Avg runtime in seconds: 0.413419508934021


,lan,cat,precision,recall,f1
0,java,summary,0.867310,0.806054,0.835561
1,java,Ownership,1.000000,1.000000,1.000000
2,java,Expand,0.318182,0.343137,0.330189
3,java,usage,0.882812,0.786543,0.831902
4,java,Pointer,0.800926,0.940217,0.865000
5,java,deprecation,1.000000,0.600000,0.750000
6,java,rational,0.260870,0.264706,0.262774
7,python,Usage,0.694215,0.694215,0.694215
8,python,Parameters,0.808000,0.789062,0.798419
9,python,DevelopmentNotes,0.294118,0.487805,0.366972


Next, the models are scored as a whole using their average F1, measured GFLOPS, and average runtime.

In [27]:
max_avg_runtime = 5
max_avg_flops = 5000

# s𝑢𝑏𝑚𝑖𝑠𝑠𝑖𝑜𝑛_𝑠𝑐𝑜𝑟𝑒(𝑚𝑜𝑑𝑒𝑙)=(𝑎𝑣𝑔. 𝐹1)×0.60+((𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)/𝑚𝑎𝑥_𝑎𝑣𝑔_𝑟𝑢𝑛𝑡𝑖𝑚𝑒)×0.2+((𝑚𝑎𝑥_GFLOPs−𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑑_GFLOPs)/𝑚𝑎𝑥_GFLOPs)×0.2
def score(avg_f1, avg_runtime, avg_flops):
    return (0.6 * avg_f1 +
      0.2 * ((max_avg_runtime - avg_runtime) / max_avg_runtime) +
      0.2 * ((max_avg_flops - avg_flops) / max_avg_flops))

avg_f1 = scores.f1.mean()
avg_runtime = total_time/10
avg_flops = total_flops/10

round(score(avg_f1, avg_runtime, avg_flops), 2)

0.75

Finally, outputting the average F1 is used to get another measure of the models.

In [28]:
print(avg_f1)

0.6371616155083775
